In [1]:
import numpy as np
import time

In [4]:
# CONSTANTS
NO_METERS_P = 108
NO_POLES_P = 117
NO_METERS_C = 7172
NO_POLES_C = 6550

In [27]:
# Preprocessing included greedy algorithm
data_file = np.loadtxt('cap360.txt', dtype=np.int32)
# Create a meters x poles matrix
Adj_pm = np.zeros((NO_METERS_C, NO_POLES_C))
# another modifiable matrix 
Mod_adj_pm = np.zeros((NO_METERS_C, NO_POLES_C))
for x in data_file:
    x[0] -= 1 # Converting to 0-index
    x[1] -= 1
    Adj_pm[x[0]][x[1]] = 1
    Mod_adj_pm[x[0]][x[1]] = 1

start_time = time.time()
# Create an zero vector for covered meters (assigned 0 for uncovered meters)
cov_meters = np.zeros(NO_METERS_C)
# list of covered meters
list_cov_meters = []
# list of covering poles
list_cov_poles = []
total = 0
nps = 3
while (np.any(cov_meters == 0)):
    # get sorting indices
    #indices = np.argsort(np.sum(Mod_adj_pm, axis = 0))[::-1]
    # Preprocessing (nps(no of preprocessing steps) times):
    # 1. Removing singleton rows
    singleton_rows = [int(x) for x in np.argwhere(np.sum(Mod_adj_pm, axis = 1) == 1)]
    # Add the pole corresponding to the first singleton row
    if singleton_rows: # if there is atleast one singleton row
        print("Singleton pole added!")
        for i in range(nps):
            pole = int(np.argwhere(Mod_adj_pm[singleton_rows[i], :] == 1))
            list_cov_poles.append(pole)        
            x = Adj_pm[:, pole]
            met_poles = [int(m) for m in np.argwhere(x == 1)] # time in orders of 10-4
            cov_meters[met_poles] += 1
            Mod_adj_pm[:, pole] = 0 
    # 2. Removing rows that contain row j
    rows_to_check = np.random.randint(0, NO_METERS_C, nps)
    for i in range(nps):
        j = rows_to_check[i]
        contains = np.argwhere(np.all(Mod_adj_pm[i] <= Mod_adj_pm, axis = 1))
        if (contains.shape[0] > 1):
            Mod_adj_pm[contains[1:], :] = 0
        '''
        if (np.sum(Mod_adj_pm[j]) == 0):
                j = j + 1
                continue
        for k in range(NO_METERS_C):
            if(np.all(Mod_adj_pm[j] <= Mod_adj_pm[k]) and j!=k): 
                print("row ", k, "contains row", j)
                Mod_adj_pm[k, :] = 0
        '''
    # 3. Removing columns contained in column j
    cols_to_check = np.random.randint(0, NO_POLES_C, nps)
    for i in range(nps):
        j = cols_to_check[i]
        contains = np.argwhere(np.all(Mod_adj_pm[:, i] >= Mod_adj_pm.T, axis = 1)).shape[0]
        if (contains.shape[0] > 1):
            Mod_adj_pm[:, contains[1:]] = 0
        
        '''
        print("checking column", j, "with sum", np.sum(Mod_adj_pm[:, j]))
        if (np.sum(Mod_adj_pm[:, j]) == 0):
                j = j + 1
                continue
        for k in range(NO_POLES_C):
            if(np.all(Mod_adj_pm[:, j] <= Mod_adj_pm[:, k]) and j!=k): 
                print("column ", j, "is contained", k)
                Mod_adj_pm[:, j] = 0
                break
        '''

    indices = np.argmax(np.sum(Mod_adj_pm, axis = 0))
    #print("best pole is ", indices)
    # add the best pole (indices)
    list_cov_poles.append(indices)
    # find the corresponding column in the matrix
    x = Adj_pm[:, indices]
    # find the meters covered by this pole and add to overall covered meters
    # and set the corresponding row of the meter to zero
    met_poles = [int(m) for m in np.argwhere(x == 1)] # time in orders of 10-4
    cov_meters[met_poles] += 1
    Mod_adj_pm[met_poles, :] = 0
    #for met_idx in met_poles:
        #cov_meters[int(met_idx)] += 1
        #Mod_adj_pm[met_idx, :] = 0
    # Clean up
    #st = time.time()
    clean_up_indices = []
    for i in range(0, len(list_cov_poles)): # for each already selected pole(sp)
        #if (i >= len(list_cov_poles)):
        #    break
        sp = list_cov_poles[i]
        #cleanable = True
        #print(meters_by_sp)
        #for x in (meters_by_sp):
        #    if ((cov_meters[int(x)] - 1) == 0):
        #        cleanable = False
        #        break
        meters_by_sp = [int(m) for m in np.argwhere(Adj_pm[:, sp] == 1)] # meters covered by sp
        if (np.any((cov_meters[meters_by_sp]-1) == 0) == False):
            clean_up_indices.append(i)
            #list_cov_poles.remove(sp)
            print("\nremoving ", sp)
            cov_meters[meters_by_sp] -= 1
            #for x in (meters_by_sp):
                #cov_meters[int(x)] -= 1
                #print(x, "   ", cov_meters[x])
    st = time.time()
    for j in sorted(clean_up_indices, reverse=True):
        del list_cov_poles[j]
    total += (time.time()-st)
    #print(time.time()-st)
    #print(cov_meters)
    print("no of poles:", len(list_cov_poles))
    print(np.sum(cov_meters != 0))
print((time.time() - start_time))
print(len(list_cov_poles))
print(total)
#print(list_cov_poles)

checking column 3874 with sum 33.0
checking column 4875 with sum 30.0
checking column 5637 with sum 10.0
no of poles: 1
57
checking column 4758 with sum 31.0
checking column 2007 with sum 25.0
checking column 545 with sum 8.0
no of poles: 2
114
checking column 4192 with sum 16.0
checking column 5136 with sum 23.0
checking column 720 with sum 16.0
no of poles: 3
168
checking column 4791 with sum 22.0
checking column 3720 with sum 17.0
checking column 788 with sum 26.0
no of poles: 4
219
checking column 3333 with sum 39.0
checking column 5217 with sum 30.0
checking column 1077 with sum 22.0
no of poles: 5
269
checking column 2442 with sum 28.0
checking column 2834 with sum 17.0
checking column 6539 with sum 9.0
no of poles: 6
319
checking column 12 with sum 5.0
checking column 5944 with sum 26.0
checking column 2924 with sum 43.0
no of poles: 7
368
checking column 715 with sum 11.0
checking column 3911 with sum 37.0
checking column 2077 with sum 38.0
no of poles: 8
417
checking column 48

KeyboardInterrupt: 

In [10]:
np.random.randint(0, 100, size=2)

array([43, 27])

In [16]:
x = np.array([[0, 0, 0], [1, 1, 1]])

In [17]:
x

array([[0, 0, 0],
       [1, 1, 1]])

In [19]:
np.all(x[0] > x[1])

False

In [23]:
for j in range(NO_POLES_C):
    print(j)
    for k in range(NO_POLES_C):
        if(np.all(Adj_pm[:, j] <= Adj_pm[:, k]) and j!=k): 
            print("column ", k, "contains column", j)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176


KeyboardInterrupt: 

In [50]:
np.all(Adj_pm[0] <= Adj_pm[2])

False

In [51]:
np.all(Adj_pm[0] <= Adj_pm, axis = 1)

array([ True, False, False, ..., False, False, False])

In [62]:
Adj_pm.T.shape

(6550, 7172)

In [66]:
Adj_pm.T[0]

2.0

In [69]:
np.argwhere(np.all(Adj_pm[:, 0] <= Adj_pm.T, axis = 1))

array([[0]])

In [56]:
for i in range(NO_POLES_C):
    print(i)
    if (np.argwhere(np.all(Adj_pm[:, i] <= Adj_pm[:, :], axis = 1)).shape[0] > 1):
        print(i, 'has')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [70]:
for i in range(NO_POLES_C):
    print(i)
    if (np.argwhere(np.all(Adj_pm[:, i] <= Adj_pm.T, axis = 1)).shape[0] > 1):
        print(i, 'has')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [84]:
x = np.array([[3, 4, 5], [2, 2, 1]])

In [85]:
x

array([[3, 4, 5],
       [2, 2, 1]])

In [86]:
x.T

array([[3, 2],
       [4, 2],
       [5, 1]])

In [87]:
x[:, 0] <= x.T

array([[ True,  True],
       [ True,  True],
       [ True, False]])

In [88]:
np.all(x[:, 0] <= x.T, axis = 1)

array([ True,  True, False])

In [92]:
np.argwhere(np.all(x[:, 0] <= x.T, axis = 1)).shape

(2, 1)